# Ruled Based Model
### Hey Together,   
Welcome to the notebook for the ruled based model of commfy.

   
## Part 1: Import libarys

In [1]:
import random
from random import randint
import requests, json
import pandas as pd
from datetime import datetime, timezone
import pytz
import numpy as np
import config

## Part 2: Gather input
There is the option to use the function input_generator() or generate_multi_input(a) whenever you need it.

In [2]:
## auto generated input:
def input_generator():
    location_list=('Berlin','Hamburg','München','Freiburg','Melbourne', 'Moskau') # example list 
    location= location_list[randint(0,5)]
    time=datetime.now(pytz.timezone('Europe/Berlin')).replace(tzinfo=None)##no timezone for better work through
    time=time.replace(hour=randint(time.hour,23))
    heaviness_of_trip=randint(1,3)
    if heaviness_of_trip==1: heaviness_of_trip="easy"
    if heaviness_of_trip==2: heaviness_of_trip="normal"
    if heaviness_of_trip==3: heaviness_of_trip="hard"
    heat_preference=randint(-5,5)
    sex=randint(1,3)
    if sex==1: sex="m"
    if sex==2: sex="f"
    if sex==3: sex="d"
    df=pd.DataFrame({
    "location" : [location],
    "time": [time], 
    "heaviness_of_trip": [heaviness_of_trip], 
    "heat_preference" : [heat_preference],
    "sex": [sex]
    })
    return df

In [3]:
#test input generator
print(input_generator().to_json())
input_generator()

{"location":{"0":"Moskau"},"time":{"0":1642370801855},"heaviness_of_trip":{"0":"easy"},"heat_preference":{"0":-5},"sex":{"0":"f"}}


,location,time,heaviness_of_trip,heat_preference,sex
0,Melbourne,2022-01-16 19:06:41.855393,hard,4,f


In [69]:
#autogenerate DataFrame with multiple input
def generate_multi_input(a):
    df=input_generator()
    for t in range(a-1): #a Trips today (it is asking for one person, so the sex, the heat preference and the ocation ist not changing)
        df=df.append(input_generator(),ignore_index = True)
        df.at[t+1,'location']=df['location'].iloc[0]
        df.at[t+1,'heat_preference']=df['heat_preference'].iloc[0]
        df.at[t+1,'sex']=df['sex'].iloc[0]
    return df

In [70]:
#test data with json input
a=generate_multi_input(4)
print(a)
a=generate_multi_input(4).to_json(orient="index",date_format='iso')
print(a)
json_inp= pd.read_json(a, orient='index',convert_dates=['time'])   
print(json_inp)

  location                       time heaviness_of_trip  heat_preference sex
0  München 2022-01-16 20:31:29.133198              easy                0   d
1  München 2022-01-16 16:31:29.135660            normal                0   d
2  München 2022-01-16 16:31:29.139014              easy                0   d
3  München 2022-01-16 22:31:29.145308              easy                0   d
{"0":{"location":"Hamburg","time":"2022-01-16T23:31:29.151Z","heaviness_of_trip":"hard","heat_preference":-1,"sex":"m"},"1":{"location":"Hamburg","time":"2022-01-16T19:31:29.152Z","heaviness_of_trip":"normal","heat_preference":-1,"sex":"m"},"2":{"location":"Hamburg","time":"2022-01-16T23:31:29.157Z","heaviness_of_trip":"hard","heat_preference":-1,"sex":"m"},"3":{"location":"Hamburg","time":"2022-01-16T17:31:29.162Z","heaviness_of_trip":"hard","heat_preference":-1,"sex":"m"}}
  location                             time heaviness_of_trip  \
0  Hamburg 2022-01-16 23:31:29.151000+00:00              hard   
1  Ha

<hr style="border:2px solid gray"> </hr>   
   
## Part 3: Data cleaning
Go through the Dataframe and check if there is non values or wild data.
1. if heat preference is empty--> 0
2. if sex is empty --> female
3. if time is empty --> now timestamp
4. if heavines of trip is empty --> normal
5. if location is empty --> berlin

<hr style="border:2px solid gray"> </hr>   
   
## Part 4: Further data collection
Weather(df) function to get from location and the time the weather data.

In [100]:
def weather(df):
    # base_url variable to store url
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    future_url="https://api.openweathermap.org/data/2.5/forecast?"
    df['weather']=None
    df['temperature']=None
    #for every row in the pandas:
    for i in range(len(df.index)) :

        # Give city name
        city_name = df['location'].iloc[i]
        df_time=df['time'].iloc[i].replace(tzinfo=None)
        now=datetime.now(pytz.timezone('Europe/Berlin')).replace(tzinfo=None)
        # complete_url variable to store
        # complete url address
        complete_current_url = base_url + "appid=" + config.api_key + "&q=" + city_name + "&units=metric"
        complete_future_url= future_url + "q=" + city_name + "&appid=" + config.api_key + "&units=metric"
        # return response object
        response = requests.get(complete_current_url).json() #response for current time
        response_future = requests.get(complete_future_url).json() # response of weather (in 3 hour timeframes)
        time_forecast=datetime.strptime(response_future['list'][0]['dt_txt'],'%Y-%m-%d %H:%M:%S')
        time_forecast2=datetime.strptime(response_future['list'][1]['dt_txt'],'%Y-%m-%d %H:%M:%S')
        # json method of response object
        # convert json format data into
        # python format data
        if (abs((df_time-now).total_seconds()) <abs((time_forecast-df_time).total_seconds()))&(abs((df_time-now).total_seconds()) <abs((time_forecast2-df_time).total_seconds())): #is the time closer to now then to any of the future forecast
            df.at[i,'weather']=response['weather'][0]['main']
            df.at[i,'temperature']=response['main']['temp']
            df.at[i,'windspeed']=response['wind']['speed']
        else:
            for a in range(10): # looks up to 30 hours in the future
                time_forecast=datetime.strptime(response_future['list'][a]['dt_txt'],'%Y-%m-%d %H:%M:%S')
                substrakt=time_forecast-df_time
                if abs(substrakt.total_seconds())<5400: ## Forecast is just every 3 hours, so it is always taking the closest one
                    df.at[i,'weather']=response_future['list'][a]['weather'][0]['main']
                    df.at[i,'temperature']=response_future['list'][a]['main']['temp']
                    df.at[i,'windspeed']=response_future['list'][a]['wind']['speed']
                    break   
    return df

In [101]:
#weather test
#two dataframes (one with just 1 input, and one with 6 input Trips)
test=weather(input_generator())
len(test.index)
test.to_json()
print(test)
test2=weather(generate_multi_input(20))
test2.head()

    location                       time heaviness_of_trip  heat_preference  \
0  Melbourne 2022-01-20 20:38:39.486119              easy               -4   

  sex weather temperature  windspeed  
0   f  Clouds       24.31       3.02  


,location,time,heaviness_of_trip,heat_preference,sex,weather,temperature,windspeed
0,München,2022-01-20 13:38:39.787515,normal,4,d,Snow,0.79,5.76
1,München,2022-01-20 09:38:39.788511,easy,4,d,Rain,1.3,0.89
2,München,2022-01-20 10:38:39.791503,easy,4,d,Rain,1.3,0.89
3,München,2022-01-20 17:38:39.793498,easy,4,d,Snow,-1.63,5.25
4,München,2022-01-20 15:38:39.795493,normal,4,d,Snow,0.79,5.76


<hr style="border:2px solid gray"> </hr>   
   
## Part 5: Define the model
Function add_dummy(df) for test purposes only.<p>
Decision trees defining the output of this model.<p>
All functions have Dataframes as input and add new columns to it and return the bigger Dataframe.

In [8]:
def add_dummy(df):
    df['headwear']=1
    df['sunglasses']=0
    df['neck']=1
    df['singlet']=0
    df['shirt']=0
    df['upper_body_second_layer']=1
    df['upper_body_third_layer']=1
    df['upper_body_outerlayer']=1
    df['hands']=1
    df['legs']=1
    df['raintrousers']=0
    df['socks']=1
    df['shoes']=1
    for i in range(len(df.index)):
        df.at[i,'headwear']=randint(1,4)
        df.at[i,'sunglasses']=randint(0,1)
        df.at[i,'neck']=randint(1,3)
        df.at[i,'singlet']=randint(0,1)
        df.at[i,'shirt']=randint(0,1)
        df.at[i,'upper_body_second_layer']=randint(1,3)
        df.at[i,'upper_body_third_layer']=randint(1,3)
        df.at[i,'upper_body_outerlayer']=randint(1,5)
        df.at[i,'hands']=randint(1,3)
        df.at[i,'legs']=randint(1,2)
        df.at[i,'raintrousers']=randint(0,1)
        df.at[i,'socks']=randint(1,2)
        df.at[i,'shoes']=randint(1,3)
    return df

In [9]:
##todo implement the logic of the decision tree
##headwear decisiontree
def headwear(df):
   return df

In [10]:
## sunglasses decisiontree
def sunglasses(df):
    df['sunglasses']=0
    for i in range(len(df.index)):
        if df['weather'].iloc[i]=='Clear':
            if df['temperature'].iloc[i]>20: 
                df.at[i,'sunglasses']=1
    return df

In [11]:
## neck decision tree
def neck(df):
    
    df['neck']=3
    for i in range(len(df.index)):
        red=0.#reducing parameter
        if df['sex'].iloc[i]=='f':
            red=2.
        t_p_s=df['temperature'].iloc[i]+df['heat_preference'].iloc[i]-red
        if t_p_s>10:
            if (t_p_s<20) & (df['weather'].iloc[i]=='Rain'):
                df.at[i,'neck']=2
            else :
                df.at[i,'neck']=1
        else:
            if t_p_s>0:
                df.at[i,'neck']=2
            else:
                if df['weather'].iloc[i]=='Clear':
                    df.at[i,'neck']=2
    return df

In [12]:
##todo implement the logic of the decision tree
##singlet decisiontree
def singlet(df):
    return df

In [13]:
##todo implement the logic of the decision tree
##shirt decisiontree
def shirt(df):
    return df

In [14]:
##todo implement the logic of the decision tree
##upper_body_second_layer decisiontree
def upper_body_second_layer(df):
    return df

In [15]:
##todo implement the logic of the decision tree
##upper_body_third_layer decisiontree
def upper_body_third_layer(df):
    return df

In [16]:
##todo implement the logic of the decision tree
##upper_body_outerlayer decisiontree
def upper_body_outerlayer(df):
    return df

In [17]:
##todo implement the logic of the decision tree
##hands decisiontree
def hands(df):
    return df

In [18]:
##todo implement the logic of the decision tree
##legs decisiontree
def legs(df):
    return df

In [19]:
##todo implement the logic of the decision tree
##raintrousers decisiontree
def raintrousers(df):
    return df

In [20]:
##todo implement the logic of the decision tree
##socks decisiontree
def socks(df):
    return df

In [21]:
##todo implement the logic of the decision tree
##shoes decisiontree
def shoes(df):
    return df

In [22]:
##summing up the decision trees into 1 list
def get_clothing_output_rbm(df):
    headwear(df)
    sunglasses(df)
    neck(df)
    singlet(df)
    shirt(df)
    upper_body_second_layer(df)
    upper_body_third_layer(df)
    upper_body_outerlayer(df)
    hands(df)
    legs(df)
    raintrousers(df)
    socks(df)
    shoes(df)
    return df

In [74]:
#test the model
get_clothing_output_rbm(test2)

,location,time,heaviness_of_trip,heat_preference,sex,weather,temperature,sunglasses,neck
0,Berlin,2022-01-16 23:32:39.468444,hard,1,d,Clouds,5.23,0,2
1,Berlin,2022-01-16 16:32:39.472471,easy,1,d,Clouds,3.14,0,2
2,Berlin,2022-01-16 20:32:39.477203,normal,1,d,Clouds,4.16,0,2
3,Berlin,2022-01-16 20:32:39.482137,easy,1,d,Clouds,4.16,0,2
4,Berlin,2022-01-16 19:32:39.486417,normal,1,d,Clouds,4.16,0,2
5,Berlin,2022-01-16 19:32:39.490742,hard,1,d,Clouds,4.16,0,2
6,Berlin,2022-01-16 20:32:39.494892,normal,1,d,Clouds,4.16,0,2
7,Berlin,2022-01-16 23:32:39.498971,hard,1,d,Clouds,5.23,0,2
8,Berlin,2022-01-16 22:32:39.503862,easy,1,d,Clouds,5.23,0,2
9,Berlin,2022-01-16 23:32:39.508160,easy,1,d,Clouds,5.23,0,2


In [95]:
add_dummy(test2)

,location,time,heaviness_of_trip,heat_preference,sex,weather,temperature,sunglasses,neck,headwear,singlet,shirt,upper_body_second_layer,upper_body_third_layer,upper_body_outerlayer,hands,legs,raintrousers,socks,shoes
0,Berlin,2022-01-16 23:32:39.468444,hard,1,d,Clouds,4.63,0,2,4,0,0,3,2,5,2,1,0,2,1
1,Berlin,2022-01-16 16:32:39.472471,easy,1,d,Clouds,3.32,1,3,4,0,0,3,2,5,1,2,1,1,3
2,Berlin,2022-01-16 20:32:39.477203,normal,1,d,Clouds,3.84,0,3,1,0,1,1,2,5,3,1,0,2,1
3,Berlin,2022-01-16 20:32:39.482137,easy,1,d,Clouds,3.84,1,1,2,1,1,2,1,4,3,1,0,2,1
4,Berlin,2022-01-16 19:32:39.486417,normal,1,d,Clouds,3.84,0,2,2,1,0,1,3,2,3,2,1,2,2
5,Berlin,2022-01-16 19:32:39.490742,hard,1,d,Clouds,3.84,1,3,2,0,1,3,1,1,1,2,1,2,3
6,Berlin,2022-01-16 20:32:39.494892,normal,1,d,Clouds,3.84,1,1,1,1,1,1,1,5,2,1,0,2,2
7,Berlin,2022-01-16 23:32:39.498971,hard,1,d,Clouds,4.63,1,3,2,0,1,3,3,5,1,1,0,2,1
8,Berlin,2022-01-16 22:32:39.503862,easy,1,d,Clouds,4.63,0,1,4,0,1,2,1,4,2,2,1,2,3
9,Berlin,2022-01-16 23:32:39.508160,easy,1,d,Clouds,4.63,0,1,2,0,0,1,3,1,1,2,0,1,1


<hr style="border:2px solid gray"> </hr>  

## Part 6: Model evaluation
How do you evaluate the quality of the decisions? Assume you get true label values.
Which evaluation metrics do you use? Which plots would be interesting?

<hr style="border:2px solid gray"> </hr>  

## Part 7: Output concatenating
get_clothing(df) function to give out 3 Dataframes (one for wearing now, one for the backpack and one what to wear at which trip).<p>
Functions recommendation_df(df) and recommendation_json(js) are translating the input (either from a Dataframe or a Json) into a Json for the Flask API.

In [93]:
# get three lists for to wear now and to bring in the backpack and a full DF what to wear at which trip
def get_clothing(df):
    dic=['headwear','sunglasses','neck','singlet','shirt','upper_body_second_layer','upper_body_third_layer','upper_body_outerlayer','hands','legs','raintrousers','socks','shoes'] # all columns
    oneisnone=['headwear','neck','upper_body_second_layer','upper_body_third_layer','upper_body_outerlayer','hands'] # all colums where 1 is the lowest possible output
    ##wear_now is the first clothing set from the first input beside when shoes and jacket are to big, they have to be wear them 
    wear_now=df.iloc[0:1].copy()
    ##in the backpack is all the other necessary clothes
    backpack_help=df.iloc[:,7:].copy()
    backpack=backpack_help.iloc[0:0].copy() # empty DF with column names
    for i in dic:
        if backpack_help[i].unique().max()==0|((i in oneisnone) & backpack_help[i].unique().max()==1): #if the maximum of the category is not wear any, stop here
            backpack.at[0,i]=None
        else:
            z=0
            while z<backpack_help[i].nunique():
                if backpack_help[i].unique()[z]!=0: #if the the newest item of the category is not to wear any, skip it
                    if ((i in oneisnone) & backpack_help[i].unique()[z]!=1):
                        backpack.at[z,i]=backpack_help[i].unique()[z]
                z=z+1
    ##if shoes or the outerlayer are bigger in the backpack we gonna wear the biggest one, so the backpack won't get to full
    if backpack_help['shoes'].unique().max()> wear_now['shoes'][0]:
        wear_now.at[0,'shoes']=backpack_help['shoes'].unique().max()
    backpack.at[:,'shoes']=None
    if backpack_help['upper_body_outerlayer'].unique().max()> wear_now['upper_body_outerlayer'][0]:
        wear_now.at[0,'upper_body_outerlayer']=backpack_help['upper_body_outerlayer'].unique().max()
    backpack.at[:,'upper_body_outerlayer']=None  
    backpack=backpack.replace({np.nan: None})
    #Ouput of all Clothing Sets in every trip, shoes and outerlayer must be the same as in the first trip
    clothing_per_trip=df
    clothing_per_trip.at[:,'shoes']=wear_now['shoes'][0]
    clothing_per_trip.at[:,'upper_body_outerlayer']=wear_now['upper_body_outerlayer'][0]

    return wear_now, backpack, clothing_per_trip


In [26]:
def recommendation_json(js):
    json_inp= pd.read_json(js, orient='index',convert_dates=['time'])  
    rec_dfs= get_clothing(add_dummy(weather(json_inp)))
    rec_js= '{"clothes":{"now":'+rec_dfs[0].to_json(orient="records")
    rec_js= rec_js+',"backpack":'+rec_dfs[1].apply(lambda x: [x.dropna()], axis=1).to_json() # drop the nan from df
    rec_js= rec_js+',"perTrip":'+rec_dfs[2].to_json(orient="index")+'}}'
    return rec_js

In [44]:
def recommendation_df(df):
    rec_dfs= get_clothing(add_dummy(weather(df)))
    rec_js= '{"clothes":{"now":'+rec_dfs[0].to_json(orient="records")
    rec_js= rec_js+',"backpack":'+rec_dfs[1].apply(lambda x: [x.dropna()], axis=1).to_json()#(orient="columns")
    rec_js= rec_js+',"perTrip":'+rec_dfs[2].to_json(orient="index")+'}}'
    return rec_js

In [94]:
recommendation_df(test2)

'{"clothes":{"now":[{"location":"Berlin","time":1642375959468,"heaviness_of_trip":"hard","heat_preference":1,"sex":"d","weather":"Clouds","temperature":4.63,"sunglasses":0,"neck":1,"headwear":4,"singlet":0,"shirt":0,"upper_body_second_layer":3,"upper_body_third_layer":3,"upper_body_outerlayer":4,"hands":3,"legs":1,"raintrousers":0,"socks":1,"shoes":3}],"backpack":{"0":[{"headwear":4.0,"legs":1.0,"socks":1.0}],"3":[{"headwear":2.0}],"1":[{"sunglasses":1.0,"neck":2.0,"singlet":1.0,"shirt":1.0,"legs":2.0,"raintrousers":1.0,"socks":2.0}],"2":[{"upper_body_second_layer":2.0,"upper_body_third_layer":2.0,"hands":2.0}]},"perTrip":{"0":{"location":"Berlin","time":1642375959468,"heaviness_of_trip":"hard","heat_preference":1,"sex":"d","weather":"Clouds","temperature":4.63,"sunglasses":0,"neck":1,"headwear":4,"singlet":0,"shirt":0,"upper_body_second_layer":3,"upper_body_third_layer":3,"upper_body_outerlayer":4,"hands":3,"legs":1,"raintrousers":0,"socks":1,"shoes":3},"1":{"location":"Berlin","time"

In [28]:
#for testing with json input
print(a)
recommendation_json(a)

{"0":{"location":"Berlin","time":"2022-01-16T22:06:42.039Z","heaviness_of_trip":"normal","heat_preference":4,"sex":"m"},"1":{"location":"M\u00fcnchen","time":"2022-01-16T21:06:42.039Z","heaviness_of_trip":"hard","heat_preference":4,"sex":"f"},"2":{"location":"Melbourne","time":"2022-01-16T18:06:42.039Z","heaviness_of_trip":"normal","heat_preference":1,"sex":"m"},"3":{"location":"Moskau","time":"2022-01-16T20:06:42.039Z","heaviness_of_trip":"normal","heat_preference":2,"sex":"f"}}


'{"clothes":{"now":[{"location":"Berlin","time":1642370802039,"heaviness_of_trip":"normal","heat_preference":4,"sex":"m","weather":"Clouds","temperature":4.09,"headwear":2,"sunglasses":0,"neck":2,"singlet":1,"shirt":1,"upper_body_second_layer":2,"upper_body_third_layer":2,"upper_body_outerlayer":5,"hands":2,"legs":1,"raintrousers":0,"socks":1,"shoes":3}],"backpack":{"headwear":{"0":2.0,"1":4.0,"2":null},"sunglasses":{"0":0.0,"1":1.0,"2":null},"neck":{"0":2.0,"1":3.0,"2":1.0},"singlet":{"0":1.0,"1":0.0,"2":null},"shirt":{"0":1.0,"1":0.0,"2":null},"upper_body_second_layer":{"0":2.0,"1":1.0,"2":null},"upper_body_third_layer":{"0":2.0,"1":1.0,"2":null},"upper_body_outerlayer":{"0":null,"1":null,"2":null},"hands":{"0":2.0,"1":3.0,"2":null},"legs":{"0":1.0,"1":2.0,"2":null},"raintrousers":{"0":0.0,"1":1.0,"2":null},"socks":{"0":1.0,"1":2.0,"2":null},"shoes":{"0":null,"1":null,"2":null}},"perTrip":{"0":{"location":"Berlin","time":1642370802039,"heaviness_of_trip":"normal","heat_preference":4,

In [29]:
#for testing generated input
add_dummy(test2)

,location,time,heaviness_of_trip,heat_preference,sex,weather,temperature,sunglasses,neck,headwear,singlet,shirt,upper_body_second_layer,upper_body_third_layer,upper_body_outerlayer,hands,legs,raintrousers,socks,shoes
0,Moskau,2022-01-16 23:06:42.678256,easy,2,f,Clouds,-7.25,0,1,1,1,0,1,1,5,2,2,0,2,3
1,Melbourne,2022-01-16 18:06:42.678256,easy,4,m,Rain,19.09,0,1,3,0,1,1,2,2,3,2,1,1,2
2,Berlin,2022-01-16 19:06:42.678256,easy,-4,m,Clouds,3.31,1,3,3,0,1,3,1,3,1,1,1,2,2
3,Berlin,2022-01-16 16:06:42.686314,easy,5,m,Clouds,2.97,0,2,4,0,0,3,2,2,1,1,0,1,1
4,Melbourne,2022-01-16 20:06:42.686314,normal,3,f,Clouds,18.76,1,3,4,0,0,1,1,5,3,1,1,1,2
5,Melbourne,2022-01-16 18:06:42.686314,normal,1,f,Rain,19.09,0,1,1,0,1,1,2,3,2,2,0,1,2
6,Moskau,2022-01-16 20:06:42.694339,easy,4,d,Clouds,-7.17,1,3,3,0,0,3,3,4,2,2,1,1,1
7,Hamburg,2022-01-16 19:06:42.702272,hard,-1,m,Clouds,5.25,1,1,2,0,1,3,1,2,3,2,0,2,3
8,Moskau,2022-01-16 17:06:42.710295,easy,0,d,Clouds,-6.62,1,2,4,0,0,1,2,2,2,2,0,1,1
9,München,2022-01-16 19:06:42.710295,easy,-4,d,Clouds,2.28,0,3,1,0,1,2,2,3,2,1,1,1,1


In [30]:
print(get_clothing(test2))

print(get_clothing(test2)[0].to_json(orient="records"))
print(get_clothing(test2)[1].to_json(orient="columns"))
print(get_clothing(test2)[2].to_json(orient="index"))

# res = clothing_per_trip.to_json(orient="records")
#  parsed = json.loads(res)
#   print(json.dumps(parsed,indent=1))

(  location                       time heaviness_of_trip  heat_preference sex  \
0   Moskau 2022-01-16 23:06:42.678256              easy                2   f   

  weather temperature  sunglasses  neck  headwear  singlet  shirt  \
0  Clouds       -7.25           0     1         1        1      0   

   upper_body_second_layer  upper_body_third_layer  upper_body_outerlayer  \
0                        1                       1                      5   

   hands  legs  raintrousers  socks  shoes  
0      2     2             0      2      3  ,   sunglasses  neck headwear singlet shirt upper_body_second_layer  \
0        0.0   1.0      1.0     1.0   0.0                     1.0   
1        1.0   3.0      3.0     0.0   1.0                     3.0   
2       None   2.0      4.0    None  None                     2.0   
3       None  None      2.0    None  None                    None   
4       None  None     None    None  None                    None   

  upper_body_third_layer upper_body_ou